In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [2]:
background=None

accumulated_weight=0.5

roi_top=20
roi_bottom=300
roi_right=300
roi_left=600

In [3]:
def cal_accumulated_avg(frame,accumulated_weight):
    global background
    if background is None:
        background=frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame,background,accumulated_weight)
    

In [4]:
def segment(frame,threshold=25):
    diff=cv2.absdiff(background.astype('uint8'),frame)
    ret,thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    contours,hierarchy=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) ==0:
        return 0
    else:
        hand_segment=max(contours,key=cv2.contourArea)
        return(thresholded,hand_segment)

In [5]:
def count_fingers(thresholded,hand_segment):
    conv_hull=cv2.convexHull(hand_segment)
    #TOP
    top=tuple(conv_hull[conv_hull[:,:,1].argmin()[0]])
    bottom=tuple(conv_hull[conv_hull[:,:,1].argmax()[0]])
    left=tuple(conv_hull[conv_hull[:,:,0].argmin()[0]])
    right=tuple(conv_hull[conv_hull[:,:,0].argmax()[0]])
    
    cX=(left[0]+right[0])//2
    cY=(top[1]+bottom[1])//2
    
    dst=pairwise.euclidean_distances([cX,cY],Y=[left,right,top,bottom])[0]
    max_dst=dst.max()
    radius=int(0.9*max_dst)
    circumfrence=(2*np.pi*radius)
    circular_roi=np.zeros(thresholded[:2],dtype='uint8')
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    circular_roi=cv2.bitwise_and(thresholded,thresholded,circular_roi)
    contours,hierarchy=cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cnt=0
    for cn in contours:
        (x,y,w,h)=cv2.boundingRect(cnt)
        out_wrist=(cY+(cY*0.25))>(y+h)
        limit_pts=((circumfrence*0.25)>cnt.shape[0])
        if out_wrist and limit_pts:
            cnt+=1
    return cnt

In [6]:
cam=cv2.VideoCapture(0)
num_frame=0
while True:
    ret,frame=cam.read()
    frame_c=frame.copy()
    roi=frame[roi_top:roi_bottom,roi_right:roi_left]
    grimg=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    grimg=cv2.GaussianBlur(grimg,(7,7),0)
    if num_frame<60:
        cal_accumulated_avg(grimg,accumulated_weight)
        if num_frame<=59:
            cv2.putText(frame_c,"WAIT.GETTING BACKGROUND",(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow("Finger Count",frame_c)
            
    else:
        hand=segment(grimg)
        if hand is not None:
            (thresholded,hand_segment)=hand
            cv2.drawContours(frame_c,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            fingers=count_fingers(thresholded,hand_segment)
            cv2.putText(frame_c,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow("Thresholded",thresholded)
            
            
    cv2.rectangle(frame_c,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frame+=1
    
    cv2.imshow("Finger Count",frame_c)
    k=cv2.waitKey(1) & 0xFF
    if k==27:
        break
        
        
        
cam.release()
cv2.destroyAllWindows()    

TypeError: cannot unpack non-iterable int object